In [1]:
import tensorflow as tf

print("NumPy version: ", tf.__version__)

if tf.test.is_gpu_available():
    print("GPU is available")
else:
    print("GPU is not available")
print(tf.config.list_physical_devices('GPU'))


NumPy version:  2.10.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is not available
[]


In [2]:
import logging
import time

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

# conda install matplotlib
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
import tensorflow as tf

# https://github.com/sun1638650145/Libraries-and-Extensions-for-TensorFlow-for-Apple-Silicon/releases
# python -m pip install --ignore-installed ~/Downloads/tensorflow_text-2.12.0-cp38-cp38-macosx_11_0_arm64.whl
import tensorflow_text

### Tensor Constant

In [9]:
num_samples = 10
feature_dim = 3
random_features = np.random.randn(num_samples, feature_dim)
random_labels = np.random.randint(0, 2, size=(num_samples, 1))

features = tf.constant(random_features, dtype=tf.float32)
labels = tf.constant(random_labels, dtype=tf.float32)
dataset = tf.data.Dataset.from_tensor_slices((features, labels))
dataset


<TensorSliceDataset element_spec=(TensorSpec(shape=(3,), dtype=tf.float32, name=None), TensorSpec(shape=(1,), dtype=tf.float32, name=None))>

In [43]:
features = tf.constant([
    [1, 1, 1],
    [1, 1, 1],
    [1, 1, 1],
    [1, 1, 2],
    [1, 1, 2],
    [1, 1, 2],
    [1, 2, 2],
    [1, 2, 2],
    [1, 2, 2],
    [2, 2, 2],
    [2, 2, 2],
    [2, 2, 2],
], dtype=tf.float32)
# higher value means stronger signal from relu
X = tf.fill(dims=(12,3),value=3.)-features
# price: $0.0, $1.0, $2.0
price_labels = tf.constant([
    [0],
    [1],
    [2],
    [0],
    [1],
    [2],
    [0],
    [1],
    [2],
    [0],
    [1],
    [2],
], dtype=tf.int32)
labels = tf.constant([
    [1],
    [1],
    [1],
    [1],
    [1],
    [0],
    [1],
    [0],
    [0],
    [0],
    [0],
    [0],
], dtype=tf.float32)

In [44]:
one_hot_price_labels = tf.one_hot(tf.squeeze(price_labels, axis=-1), depth=3)
print(one_hot_price_labels.shape)
one_hot_result_label = tf.tile(labels, [1,3])
print(one_hot_result_label.shape)

(12, 3)
(12, 3)


In [52]:
# delta prob $0~$1, prob $1~$2, prob $2+
bias_prob = tf.constant([
], dtype=tf.float32)
delta_price_prob = tf.constant([
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
], dtype=tf.float32)
price_prob = tf.math.cumsum(tf.math.softmax(delta_price_prob, axis=-1), axis=-1, reverse=True)
price_prob

<tf.Tensor: shape=(12, 3), dtype=float32, numpy=
array([[1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334]], dtype=float32)>

In [ ]:
# delta prob $0~$1, prob $1~$2, prob $2+
bias_prob = tf.constant([
], dtype=tf.float32)
delta_price_prob = tf.constant([
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
], dtype=tf.float32)
price_prob = tf.math.cumsum(tf.math.softmax(delta_price_prob, axis=-1), axis=-1, reverse=True)
price_prob

<tf.Tensor: shape=(12, 3), dtype=float32, numpy=
array([[1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334]], dtype=float32)>

In [48]:
delta_price_prob

<tf.Tensor: shape=(12, 3), dtype=float32, numpy=
array([[0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334]], dtype=float32)>

In [46]:
X

<tf.Tensor: shape=(12, 3), dtype=float32, numpy=
array([[2., 2., 2.],
       [2., 2., 2.],
       [2., 2., 2.],
       [2., 2., 1.],
       [2., 2., 1.],
       [2., 2., 1.],
       [2., 1., 1.],
       [2., 1., 1.],
       [2., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]], dtype=float32)>